In [1]:
import matplotlib.pyplot as plt

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer
from sklearn.model_selection import StratifiedKFold
import pickle
from numpy.random import choice
import os.path

import tensorflow as tf
import numpy as np

In [2]:
tf.enable_eager_execution()

#### Import model, training function 

In [3]:
from annsa.model_classes import (dae_model_features,
                                 DAE,
                                 dnn_model_features,
                                 DNN,
                                 save_model,
                                 train_earlystop)

#### Load Dataset

In [4]:
all_spectra = np.load('./hyperparameter_search_datasets/spectra_SBratio13_CalRange0911_3625.npy')
all_keys = np.load('./hyperparameter_search_datasets/keys_SBratio13_CalRange0911_3625.npy')

skf = StratifiedKFold(n_splits=5, random_state=5)

#### Define scaling pipeline

In [5]:
def identity_function(x):
    return x

In [6]:
dae_scaler = make_pipeline(FunctionTransformer(np.log1p, validate=True))
mlb = LabelBinarizer()

## Train network

In [7]:
number_hyperparameters_to_search = 256
earlystop_errors_test = []
model_id='dae-dnn'

#### Load past errors, if they exists

In [8]:
file_path = './hyperparameter-search-results/final_test_errors_'+model_id+'.npy'
if os.path.exists(file_path):
    earlystop_errors_test = np.load(file_path)
    start_iteration = earlystop_errors_test.shape[0]
else:
    start_iteration = 0

In [10]:
for hyperparameter_index in range(start_iteration,
                                  number_hyperparameters_to_search):
    print("\nSearching hyperparameter combination number "+
          str(hyperparameter_index))
    
    ##########################
    ### Define Autoencoder ###
    ##########################
    number_layers = choice([1,2])
    dense_nodes = 2**choice([5, 6, 7, 8, 9],size=(number_layers))
    dense_nodes = np.sort(dense_nodes)
    dense_nodes_decoder = np.sort(dense_nodes)
    dense_nodes_encoder = np.flipud(dense_nodes)
    # remove encoding from decoder
    dense_nodes_decoder = dense_nodes_decoder[1:]
    dae_features = dae_model_features(
        learining_rate=10**np.random.uniform(-4,-1),
        l1_regularization_scale=10**np.random.uniform(-3,-1),
        dropout_probability=np.random.uniform(0,1),
        batch_size=2**np.random.randint(4,10),
        dense_nodes_encoder=dense_nodes_encoder,
        dense_nodes_decoder=dense_nodes_decoder,
        scaler=dae_scaler,
        activation_function=choice([tf.nn.relu,tf.nn.tanh,tf.nn.sigmoid]),
        output_function=None)
    # Save DAE features
    with open('./hyperparameter-search-results/' + model_id + '_' +
              str(hyperparameter_index) + '_dae_features', 'wb+') as f:
        pickle.dump(dae_features,f)

    ##########################
    ## Define Dense Network ##
    ##########################
    number_layers = choice([1,2])
    dense_nodes = (10**np.random.uniform(1,
                                         np.log10(dense_nodes_encoder[-1]),
                                         number_layers)).astype('int')
    dense_nodes = np.sort(dense_nodes)
    dense_nodes = np.flipud(dense_nodes)

    dnn_features = dnn_model_features(
        learining_rate=10**np.random.uniform(-4,-1),
        l2_regularization_scale=10**np.random.uniform(-3,-1),
        dropout_probability=np.random.uniform(0,1),
        batch_size=2**np.random.randint(4,10),
        output_size=29,
        dense_nodes=dense_nodes,
        activation_function=choice([tf.nn.relu,tf.nn.tanh,tf.nn.sigmoid]),
        scaler=None)
    # Save DNN features
    with open('./hyperparameter-search-results/' + model_id + '_' +
              str(hyperparameter_index) + '_dnn_features', 'wb+') as f:
        pickle.dump(dae_features,f)
        
    ##########################
    ##### Define Training ####
    ##########################
    dae_optimizer = tf.train.AdamOptimizer(dae_features.learining_rate)
    dnn_optimizer = tf.train.AdamOptimizer(dnn_features.learining_rate)
    
    k_folds_errors = []
    not_learn_flag = 0
    train_autoencoder = True
    for train_index, test_index in skf.split(all_spectra, all_keys):
        
        if not_learn_flag:
            break

        all_keys_binarized = mlb.fit_transform(all_keys.reshape([all_keys.shape[0],1]))

        # reset model on each iteration
        dae_model = DAE(dae_features)
        dae_model.scaler.fit(all_spectra[train_index])
        dnn_model = DNN(dnn_features)
        
        ##########################
        #### Train Autoencoder ###
        ##########################
        not_learning_threshold=1.0
        if train_autoencoder:
            print('Searching Autoencoder...')
            costfunction_errors_dae_tmp, earlystop_errors_dae_tmp = train_earlystop(
                training_data=all_spectra[train_index],
                training_keys=all_spectra[train_index],
                testing_data=all_spectra[test_index],
                testing_keys=all_spectra[test_index],
                model=dae_model,
                optimizer=dae_optimizer,
                num_epochs=200,
                obj_cost=dae_model.mse,
                earlystop_cost_fn=dae_model.mse,
                earlystop_patience=10,
                not_learning_patience=10,
                not_learning_threshold=not_learning_threshold,
                verbose=True,
                fit_batch_verbose=5,
                data_augmentation=dae_model.poisson_data_augmentation)
        # only train autoencoder once
        train_autoencoder = False
        # Check if network isn't learning
        if earlystop_errors_dae_tmp[0] > not_learning_threshold:
            not_learn_flag = 1
            k_folds_errors.append(1.0)
            print('Autoencoder doesn\'t learn')
            break
        ##########################
        ### Train Dense Netowrk ##
        ##########################
        # reset DNN
        dnn_model = DNN(dnn_features)
        not_learning_threshold = 0.9
        # Pre-process using DAE
        def dae_encoding(input_data):
            return dae_model.encoder(input_data, training=False)        
        dnn_model.scaler = make_pipeline(FunctionTransformer(dae_encoding, validate=True))
        print('Searching Dense Network...')
        costfunction_errors_tmp, earlystop_errors_tmp = train_earlystop(
            training_data=all_spectra[train_index],
            training_keys=all_keys_binarized[train_index],
            testing_data=all_spectra[test_index],
            testing_keys=all_keys_binarized[test_index],
            model=dnn_model,
            optimizer=dnn_optimizer,
            num_epochs=200,
            obj_cost=dnn_model.cross_entropy,
            earlystop_cost_fn=dnn_model.f1_error,
            earlystop_patience=10,
            not_learning_patience=10,
            not_learning_threshold=not_learning_threshold,
            verbose=True,
            fit_batch_verbose=5,
            data_augmentation=dnn_model.default_data_augmentation)
        k_folds_errors.append(earlystop_errors_tmp)
        # Check if network isn't learning
        if earlystop_errors_tmp[0] > not_learning_threshold:
            not_learn_flag = 1
            print('Dense network doesn\'t learn')
            break
        
        
    earlystop_errors_average_kfolds = np.average(k_folds_errors)
    earlystop_errors_test = np.append(earlystop_errors_test,earlystop_errors_average_kfolds)
    np.save('./hyperparameter-search-results/final_test_errors_'+
            model_id, earlystop_errors_test)


Searching hyperparameter combination number 0
Searching Autoencoder...
Epoch 5: CostFunc loss: 8.64 8.63, EarlyStop loss: 6.27 6.26
Epoch 10: CostFunc loss: 12.43 12.41, EarlyStop loss: 10.06 10.04
Test error at early stop: Objectives fctn: 9.47 Early stop fctn: 9.47
Autoencoder doesn't learn

Searching hyperparameter combination number 1
Searching Autoencoder...
Epoch 5: CostFunc loss: 10.51 10.49, EarlyStop loss: 10.43 10.41
Epoch 10: CostFunc loss: 9.39 9.36, EarlyStop loss: 9.31 9.28


KeyboardInterrupt: 